In [70]:
#Import things
import astropy
import astroquery
from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle
import astropy.units as u
import numpy as np

In [69]:
# Gaia Query

coord = SkyCoord(ra=280, dec=-60, unit=(u.degree, u.degree), frame='icrs')
width = u.Quantity(0.1, u.deg)
height = u.Quantity(0.1, u.deg)
r = Gaia.query_object_async(coordinate=coord, width=width, height=height)


HTTPError: OK

In [75]:
# ATNF Query
from psrqpy import QueryATNF
query = QueryATNF(condition= 'PMRA>0 && PMDEC>0')
table = query.table
# print(table)

In [ ]:
print(table.info)

<Table length=1>
    name      dtype       unit         class     n_bad
------------ ------- -------------- ------------ -----
        PSRJ   str10                      Column     0
    PSRJ_REF    str6                      Column     0
         RAJ   str17                      Column     0
     RAJ_ERR float64                      Column     0
     RAJ_REF    str7                      Column     0
        DECJ   str18                      Column     0
    DECJ_ERR float64                      Column     0
    DECJ_REF    str7                      Column     0
    POSEPOCH float64              d       Column     0
POSEPOCH_REF    str7                      Column     0
          F0 float64             Hz       Column     0
      F0_ERR float64             Hz       Column     0
      F0_REF    str7                      Column     0
          F1 float64         1 / s2       Column     0
      F1_ERR float64         1 / s2       Column     0
      F1_REF    str7                      Column

Given an item from ATNF catalog, find nearby Gaia objects

In [72]:
from astropy.time import Time

# Gather information on a given pulsar

pulsar = 0 # check first pulsar listed in ATNF

p_ascension = table["RAJ"][pulsar]
p_declination = table["DECJ"][pulsar]
p_pmra = table["PMRA"][pulsar]
p_pmdec = table["PMDEC"][pulsar]

print (p_ascension + " " + p_declination)

raw_epoch = table["POSEPOCH"][pulsar]
p_epoch = Time(raw_epoch, format='mjd').jyear

print(p_epoch) # testing that p_epoch is now in the correct time units, jyear


10:12:33.43820707 +53:07:02.24902675
2011.0102669404519


In [ ]:
p_ascension_angle = Angle(p_ascension.tolist(), u.degree) # convert p_ascension to Angle object 
p_declination_angle = Angle(p_declination.tolist(), u.degree) # convert p_declination to Angle object

print(type(p_declination))
print(type(p_epoch.tolist()))


<class 'numpy.str_'>
<class 'float'>


In [ ]:
gaia_epoch = 2015.5 * u.yr # define gaia epoch in units years
year_diff = gaia_epoch - p_epoch.tolist() * u.yr # define the time difference in years as a float

p_pmra_deg = (p_pmra.tolist() * u.mas).to(u.deg) / u.yr # convert p_pmra from mas/yr to deg/yr
p_pmdec_deg = (p_pmdec.tolist() * u.mas).to(u.deg) / u.yr # convert p_pmdec_deg from mas/yr to deg/yr

assert p_pmra_deg.unit.is_equivalent(u.deg / u.yr)
assert p_pmdec_deg.unit.is_equivalent(u.deg / u.yr)
assert year_diff.unit.is_equivalent(u.yr)

In [ ]:
# Update location of pulsar based on epoch difference with Gaia

gaia_epoch = 2015.5 * u.yr # define gaia epoch in units years
year_diff = gaia_epoch - p_epoch.tolist() * u.yr # define the time difference in years as a float

In [ ]:
p_new_ascension = p_ascension_angle + (p_pmra_deg * year_diff)
p_new_declination = p_declination_angle + (p_pmdec_deg * year_diff)

print(p_ascension)
print(p_new_ascension)
print(p_declination)
print(p_new_declination)


10:12:33.43820707
10d12m33.45005113s
+53:07:02.24902675
53d07m02.13436937s


In [ ]:
print(table['JNAME'][pulsar])

J1012+5307


In [73]:
# Querying Gaia

coord = SkyCoord(ra=p_new_ascension, dec=p_new_declination, unit=(u.degree, u.degree), frame='icrs')
width = u.Quantity(10, u.mas)
height = u.Quantity(10, u.mas)
results = Gaia.query_object_async(coordinate=coord, width=width, height=height)
results.show_in_notebook()

HTTPError: OK